functions - not oneliners
math - not just functions

In [28]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, MinMaxScaler
warnings.filterwarnings('ignore')
data = pd.read_csv('''C:\\Users\\user\\Documents\\GitHub\\notebooks-on-ml\\Coursera lil proj\\DATA\\a_steam_data_2021_2025.csv''')

In [29]:
data['genres'] = data['genres'].fillna(data.groupby('categories')['genres'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan))
data['genres'] = data['genres'].replace(['nan', 'None', ''], np.nan)
data['categories'] = data['categories'].replace(['nan', 'None', ''], np.nan)
data = data.dropna(subset=['genres', 'categories'])
data['developer'] = data['developer'].fillna('Noname')
data['publisher'] = data['publisher'].fillna('Noname')

In [30]:
data['release_date'] = pd.to_datetime(data['release_date'], format='mixed', errors='coerce')
data['release_date'] = data['release_date'].fillna(pd.to_datetime('Aug 1 ' + data['release_year'].astype(str), format='%b %d %Y', errors='coerce'))       
data['genres'] = data['genres'].astype(str).str.split(';')
data['categories'] = data['categories'].astype(str).str.split(';')
data = data.sort_values(['release_date', 'developer']).reset_index(drop=True)

In [31]:
data['is_sequel'] = data['name'].apply(lambda x: 1 if any(word.upper() in ['2', '3', 'II', 'III', 'IV', 'V'] for word in str(x).split()) else 0).astype(int)
data['developer&publisher'] = [1 if any(i in p for i in d) else 0 for d, p in zip(data['developer'], data['publisher'])]

data['genre_count'] = data['genres'].apply(len)
data['categories_count'] = data['categories'].apply(len)

data['niche'] = data['genres'].str[0] + '_' + data['categories'].str[0]
dev_gen_seen, dev_cat_seen = {}, {}
dg_hist, dc_hist = [], []

dev_gen_seen, dev_cat_seen = {}, {}
pub_gen_seen, pub_cat_seen = {}, {}
dg_hist, dc_hist, pg_hist, pc_hist = [], [], [], []

for d, p, g, c in zip(data['developer'], data['publisher'], data['genres'], data['categories']):
    d_key = tuple(d) if isinstance(d, list) else d
    p_key = tuple(p) if isinstance(p, list) else p
    
    dg_hist.append(list(dev_gen_seen.get(d_key, [])))
    dc_hist.append(list(dev_cat_seen.get(d_key, [])))
    pg_hist.append(list(pub_gen_seen.get(p_key, [])))
    pc_hist.append(list(pub_cat_seen.get(p_key, [])))
    
    dev_gen_seen.setdefault(d_key, set()).update(g)
    dev_cat_seen.setdefault(d_key, set()).update(c)
    pub_gen_seen.setdefault(p_key, set()).update(g)
    pub_cat_seen.setdefault(p_key, set()).update(c)

data['developer_genres'] = dg_hist
data['developer_categories'] = dc_hist
data['publisher_genres'] = pg_hist
data['publisher_categories'] = pc_hist

In [32]:
data['log_price'] = np.log1p(data['price'])
data['log_recommendations'] = np.log1p(data['recommendations'])

In [33]:
data['is_hit'] = (data['log_recommendations'] > 800).astype(int)
#data['is_mega_hit'] = data['log_recommendations'] > 1800
data['is_discount'] = (data['log_price'] % 1 > 0.90).astype(int)
data['release_month'] = data['release_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['release_month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['release_month'] / 12)
data['genre_count'] = data['genres'].apply(len)
data['categories_count'] = data['categories'].apply(len)
data['niche'] = data['genres'].str[0] + '_' + data['categories'].str[0]
#data['price_per_genre'] = data['log_price'] / (data['genre_count'] + 1)

dev_stats = {}
pub_stats = {}

res = {k: [] for k in [
    'dev_hit_ratio', 'pub_hit_ratio', 'dev_avg_price', 'pub_avg_price',
    'dev_std_price', 'pub_std_price', 'dev_avg_recs', 'pub_avg_recs',
    'dev_mom', 'pub_mom', 'dev_gen_h', 'dev_cat_h', 'pub_gen_h', 'pub_cat_h'
]}

for d, p, h, pr, rec, g, c in zip(data['developer'], data['publisher'], data['is_hit'], 
                                  data['log_price'], data['log_recommendations'], data['genres'], data['categories']):
    
    for ent, stats, pref in [(d, dev_stats, 'dev'), (p, pub_stats, 'pub')]:
        if ent not in stats:
            stats[ent] = [0, 0, 0.0, 0.0, 0.0, 0.0, set(), set()]
        
        s = stats[ent]
        n = s[1]

        res[f'{pref}_hit_ratio'].append(s[0]/n if n>0 else 0)
        res[f'{pref}_avg_price'].append(s[2]/n if n>0 else 0)
        res[f'{pref}_avg_recs'].append(s[4]/n if n>0 else 0)
        res[f'{pref}_mom'].append(rec - s[5] if n>0 else 0)
        res[f'{pref}_gen_h'].append(list(s[6]))
        res[f'{pref}_cat_h'].append(list(s[7]))
        
        if n > 1:
            var = (s[3] / n) - (s[2] / n)**2
            res[f'{pref}_std_price'].append(np.sqrt(max(0, var)))
        else:
            res[f'{pref}_std_price'].append(0)

        s[0] += h
        s[1] += 1
        s[2] += pr
        s[3] += pr**2
        s[4] += rec
        s[5] = rec
        s[6].update(g)
        s[7].update(c)

data['developer_hit_ratio'] = res['dev_hit_ratio']
data['publisher_hit_ratio'] = res['pub_hit_ratio']
data['developer_price'] = res['dev_avg_price']
data['publisher_price'] = res['pub_avg_price']
data['developer_std'] = res['dev_std_price']
data['publisher_std'] = res['pub_std_price']
data['developer_recommendations'] = res['dev_avg_recs']
data['publisher_recommendations'] = res['pub_avg_recs']
data['developer_momentum'] = res['dev_mom']
data['publisher_momentum'] = res['pub_mom']

#data['unique_genres'] = [len(x) for x in res['dev_gen_h']]
#data['unique_categories'] = [len(x) for x in res['dev_cat_h']]

#data['developer_genre_overlap'] = [len(set(g) & set(h))/len(g) if len(g)>0 and h else 0 for g, h in zip(data['genres'], res['dev_gen_h'])]
#data['developer_category_overlap'] = [len(set(c) & set(h))/len(c) if len(c)>0 and h else 0 for c, h in zip(data['categories'], res['dev_cat_h'])]
#data['publisher_genre_overlap'] = [len(set(g) & set(h))/len(g) if len(g)>0 and h else 0 for g, h in zip(data['genres'], res['pub_gen_h'])]
#data['publisher_category_overlap'] = [len(set(c) & set(h))/len(c) if len(c)>0 and h else 0 for c, h in zip(data['categories'], res['pub_cat_h'])]

data['developer_game_number'] = data.groupby('developer').cumcount()
data['developer_game_number'] = data.groupby('developer')['developer_game_number'].shift(1).fillna(0).astype(int)
data['publisher_game_number'] = data.groupby('publisher').cumcount()
data['publisher_game_number'] = data.groupby('publisher')['publisher_game_number'].shift(1).fillna(0).astype(int)

data['publisher_niche_count'] = data.groupby(['publisher', 'niche']).cumcount()
data['publisher_niche_count'] = data.groupby(['publisher', 'niche'])['publisher_niche_count'].shift(1).fillna(0).astype(int)
data['developer_niche_count'] = data.groupby(['developer', 'niche']).cumcount()
data['developer_niche_count'] = data.groupby(['developer', 'niche'])['developer_niche_count'].shift(1).fillna(0).astype(int)

data['is_new_niche'] = ~data.duplicated(['developer', 'niche'])
data['total_niches_count'] = data.groupby('developer')['is_new_niche'].cumsum()
data['total_niches_count'] = data.groupby('developer')['total_niches_count'].shift(1).fillna(0).astype(int)
data['is_specialist'] = (data['total_niches_count'] <= 2).astype(int)
#data['is_veteran'] = (data['developer_niche_count'] >= 3).astype(int)

#data['timeliness'] = data['recommendations'] / ((2026 - data['release_date'].astype(str).str.split('-').str[0].astype(int)) * 12 - (data['release_date'].astype(str).str.split('-').str[1].astype(int) - 1)).replace(0, 1)
#data['quality'] = data['log_recommendations'] / (data['log_price'] + 0.001)
#data['developer_quality_ratio'] = data.groupby('developer')['quality'].transform(lambda x: x.shift(1).expanding().mean())
#data['publisher_quality_ratio'] = data.groupby('publisher')['quality'].transform(lambda x: x.shift(1).expanding().mean())
#data['developer_niche_recommendations'] = data.groupby(['developer', 'niche'])['log_recommendations'].transform(lambda x: x.shift(1).expanding().mean())
#data['publisher_niche_recommendations'] = data.groupby(['publisher', 'niche'])['log_recommendations'].transform(lambda x: x.shift(1).expanding().mean())

In [34]:
data['log_developer_price'] = np.log1p(data['developer_price'])
#data['log_publisher_price'] = np.log1p(data['publisher_price'])
data['log_developer_recommendations'] = np.log1p(data['developer_recommendations'])
data['log_publisher_recommendations'] = np.log1p(data['publisher_recommendations'])
#data['log_quality'] = np.log1p(data['quality'])
#data['log_developer_quality_ratio'] = np.log1p(data['developer_quality_ratio'])
#data['log_publisher_quality_ratio'] = np.log1p(data['publisher_quality_ratio'])
data['log_developer_momentum'] = np.sign(data['developer_momentum']) * np.log1p(np.abs(data['developer_momentum']))
#data['log_developer_niche_recommendations'] = np.log1p(data['developer_niche_recommendations'])
data['log_publisher_momentum'] = np.sign(data['publisher_momentum']) * np.log1p(np.abs(data['publisher_momentum']))
#data['log_publisher_niche_recommendations'] = np.log1p(data['publisher_niche_recommendations'])
data['log_total_niches_count'] = np.log1p(data['total_niches_count'])
data['log_developer_game_number'] = np.log1p(data['developer_game_number'])
data['log_publisher_game_number'] = np.log1p(data['publisher_game_number'])
data['log_developer_std'] = np.log1p(data['developer_std']) 
data['log_publisher_std'] = np.log1p(data['publisher_std'])

data['developer_game_number'] = data.groupby('developer').cumcount() + 1
data['publisher_game_number'] = data.groupby('publisher').cumcount() + 1
data['log_developer_game_number'] = np.log1p(data['developer_game_number'])
data['log_publisher_game_number'] = np.log1p(data['publisher_game_number'])

In [ ]:
#inf_counts = np.isinf(data.select_dtypes(include=[np.number])).sum()
#print(inf_counts[inf_counts > 0])
#print('----------------')
#nan_counts = data.isnull().sum()
#print(nan_counts[nan_counts > 0])

Series([], dtype: int64)
----------------
Series([], dtype: int64)


In [36]:
robust_cols = [
    'log_price', 'log_developer_price',
    'log_developer_recommendations', 'log_publisher_recommendations', 
#    'log_quality', 'log_developer_quality_ratio', 'log_publisher_quality_ratio', 
#    'log_publisher_price', 'log_recommendations',
    'log_developer_momentum', 'log_publisher_momentum',
#    'log_developer_niche_recommendations', 'log_publisher_niche_recommendations',
    'developer_niche_count', 'publisher_niche_count',
#    'log_developer_count', 'log_publisher_count',
    'log_total_niches_count',
    'log_developer_std', 'log_publisher_std',
]
minmax_cols = [
#    'timeliness', 
    'genre_count', 'categories_count',  
#    'unique_genres', 'unique_categories',
#    'developer_category_overlap', 'publisher_category_overlap',
#    'developer_genre_overlap', 'publisher_genre_overlap',
    'month_sin', 'month_cos'
]
robust = RobustScaler()
data[robust_cols] = robust.fit_transform(data[robust_cols])
mms = MinMaxScaler()
data[minmax_cols] = mms.fit_transform(data[minmax_cols])

In [37]:
mlb = MultiLabelBinarizer(sparse_output=True)
genre_sparse = mlb.fit_transform(data['genres'])
genre_df = pd.DataFrame.sparse.from_spmatrix(
    genre_sparse, 
    columns=mlb.classes_, 
    index=data.index
)
category_sparse = mlb.fit_transform(data['categories'])
category_df = pd.DataFrame.sparse.from_spmatrix(
    category_sparse, 
    columns=mlb.classes_, 
    index=data.index
)
mobile_cols = [c for c in category_df.columns if 'Remote Play on Phone' in c or 'Remote Play on Tablet' in c]
data['mobile'] = category_df[mobile_cols].any(axis=1).astype('int8')
data = pd.concat([data.drop(columns=['genres', 'categories']), genre_df, category_df], axis=1)

In [38]:
data = data.drop(columns=[
    'publisher', 'developer', 'price', 'name', 'niche', 'appid', 'release_date',
    'publisher_genres', 'publisher_categories', 'developer_genres', 'developer_categories',
    'publisher_price', 'developer_price', 'developer_recommendations', 
    'publisher_recommendations', 'total_niches_count', 'developer_game_number', 
    'publisher_game_number', 'release_year', 'release_month', 'recommendations', 
    'Remote Play on Phone', 'Remote Play on Tablet', 'Online PvP', 'Family Sharing',
    'Online Co-op', 'Shared/Split Screen', 'publisher_std', 'developer_std', 'log_recommendations'
])

In [39]:
data.to_csv('''C:\\Users\\user\\Documents\\GitHub\\notebooks-on-ml\\Coursera lil proj\\DATA\\transition_file.csv''', index=False)

corr_matrix = data.select_dtypes(include=[np.number]).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(f"Redundant columns to drop: {to_drop}")

pairs = upper.unstack().dropna()
sorted_pairs = pairs.sort_values(ascending=False)
print("Top 10 most redundant pairs:")
print(sorted_pairs.head(10))

sns.scatterplot(x=data['publisher_std'], y=data['developer_std'])
plt.title("Visualizing Redundancy")
plt.show()

plt.figure(figsize=(15, 10))
subset = data.iloc[:, :30] 
sns.heatmap(subset.corr(), annot=False, cmap='coolwarm')
plt.show()

plot_data = data.explode('genres').explode('categories')
sns.barplot(data=plot_data, x='release_year', y='price')
plt.show()
sns.barplot(data=plot_data, x='release_year', y='recommendations')
plt.show()

timeline_data = plot_data.groupby(['release_year', 'genres']).size().reset_index(name='count')
timeline_data_2 = plot_data.groupby(['release_year', 'categories']).size().reset_index(name='count')
sns.lineplot(data=timeline_data, x='release_year', y='count', hue='genres')
plt.show()
sns.lineplot(data=timeline_data_2, x='release_year', y='count', hue='categories')
plt.show()

sns.barplot(data=plot_data, x='price', y='genres')
plt.show()
sns.barplot(data=plot_data, x='price', y='categories')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='genres')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='categories')
plt.show()

sns.histplot(data['price'], kde=True)
plt.show()
sns.histplot(data['recommendations'], kde=True)
plt.show()
sns.histplot(data['log_price'], kde=True)
plt.show()
sns.histplot(data['log_recommendations'], kde=True)
plt.show()

In [40]:
data = pd.read_csv('''C:\\Users\\user\\Documents\\GitHub\\notebooks-on-ml\\Coursera lil proj\\DATA\\transition_file.csv''')
X = data.drop('is_hit', axis=1)
y = data['is_hit']

In [52]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import optuna
from sklearn.model_selection import StratifiedKFold

base_models = [
    ('xgb', XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=5, use_label_encoder=False, base_score=0.5, eval_metric='logloss')),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=5))
]
up_model = LogisticRegression(C=0.1, max_iter=1000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
stack = StackingClassifier(
    estimators=base_models,
    final_estimator=up_model,
    cv=skf,
    stack_method='predict_proba',
    passthrough=False
)

multi_model = MultiOutputClassifier(stack)
y = data[['is_hit', 'is_sequel']]
multi_model.fit(X, y)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(0)

In [ ]:
from sklearn.metrics import classification_report
preds = multi_model.predict(X)
for index, name in enumerate(['is_hit', 'is_sequel']):
    print(f'f\n=== Perfomance for: {name} ===')
    print(classification_report(y.iloc[:, index], preds[:, index]))

f
=== Perfomance for: is_hit ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63157
           1       1.00      1.00      1.00      2354

    accuracy                           1.00     65511
   macro avg       1.00      1.00      1.00     65511
weighted avg       1.00      1.00      1.00     65511

f
=== Perfomance for: is_sequel ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63793
           1       1.00      1.00      1.00      1718

    accuracy                           1.00     65511
   macro avg       1.00      1.00      1.00     65511
weighted avg       1.00      1.00      1.00     65511



In [ ]:
corr = data.corr()['is_hit'].sort_values(ascending=False)
corr

is_hit                           1.000000
log_recommendations              0.667534
publisher_hit_ratio              0.354078
Steam Trading Cards              0.303358
log_publisher_recommendations    0.295808
                                   ...   
developer_niche_count           -0.031098
log_developer_game_number       -0.031859
Indie                           -0.046192
Free To Play                    -0.065472
Casual                          -0.065741
Name: is_hit, Length: 99, dtype: float64